# Numerical analysis: TP-1
<h4 align="right"> Author: <i> Hicham Janati </i></h4>

In [1]:
import numpy as np

#### 1) Integers and floating point representations
run the following cells below.

In [2]:
a = 2 * np.ones(1).astype(int)
print(f"a is equal to {a} and its type is {a.dtype}.")

a is equal to [2] and its type is int64.


In [3]:
for i in range(60, 68):
    print(f"2 ** {i} = {a ** i}")

2 ** 60 = [1152921504606846976]
2 ** 61 = [2305843009213693952]
2 ** 62 = [4611686018427387904]
2 ** 63 = [-9223372036854775808]
2 ** 64 = [0]
2 ** 65 = [0]
2 ** 66 = [0]
2 ** 67 = [0]


**Q1.  Can you explain the observed behavior ?  Propose a way to fix this.**

Overflow car les nombres entiers sont ici représentés sur 64 bits (donc 63 bits pour la valeur car il y a le bit de poids fort qui est le bit du signe). Pour éviter le problème, on peut choisir d'augmenter la mémoire allouée pour pouvoir écrire les nombres sur plus de bits.

**Q2. Does the problem occur without specifying the dtype `np.ones(1)`? Deduce a real numpy usecase where this might be a problem.**

 In deep learning applications, chosing the "right" dtype is a very important tradeoff between speed and accuracy.

#### 2) Imperfect floating point numbers 

Consider the function $f(x) = 2x$ on $[0, 0.5]$ and $f(x) = 2x - 1$ on $]0.5, 1]$ 

**Q1.** Consider the sequence defined by $x_{n+1} = f(x_n)$ with $x_0 = 0.1$ Compute the first 5 elements of the sequence (manually). What do you conclude ?

In [4]:
x = 0.1
print(f"x is equal to {x} and its type is {type(x)}.")
for k in range(5):
    if x > 0.5:
        x = 2 * x - 1
    else:
        x = 2 * x
    print(f"x is equal to {x} and its type is {type(x)}.")
    

x is equal to 0.1 and its type is <class 'float'>.
x is equal to 0.2 and its type is <class 'float'>.
x is equal to 0.4 and its type is <class 'float'>.
x is equal to 0.8 and its type is <class 'float'>.
x is equal to 0.6000000000000001 and its type is <class 'float'>.
x is equal to 0.20000000000000018 and its type is <class 'float'>.


**Q2.** Complete the function below that returns $x_n$. What do you observe ?

In [5]:
x0 = 0.1

def f(x, n=100):
    for k in range(n+1):
       if x > 0.5:
        x = 2 * x - 1
    else:
        x = 2 * x 
    return x

f(x0)

0.2

`float64` numbers are represented using 64 bits as:
$$(-1)^s \quad 0.m_1..m_{52} \quad 2^{e_1..e_{11}}$$
where $s$ is a sign bit, $m$ is the mantissa (52 bits) and $e$ is the exponent (11 bits)

**Q4** Take a moment a contemplate this mystery. Use the `pretty_float_bits` function below to find an explanation for this.

In [6]:
0.1 + 0.2

0.30000000000000004

In [7]:
import struct

def float_to_bin(f) -> str:
    fmt = ">d"
    bz = struct.pack(fmt, f)
    return "".join(f"{b:08b}" for b in bz)

def sign_exponent_fraction(s):
    return s[0:1], s[1:12], s[12:64]

def pretty_float_bits(f) -> str:
    return " ".join(sign_exponent_fraction(float_to_bin(f)))

pretty_float_bits(0.1)

'0 01111111011 1001100110011001100110011001100110011001100110011010'

In [8]:
pretty_float_bits(0.1 + 0.2)

'0 01111111101 0011001100110011001100110011001100110011001100110100'

In [9]:
pretty_float_bits(0.3)

'0 01111111101 0011001100110011001100110011001100110011001100110011'

With floats, the order matters:

In [10]:
100. - 100. + 0.1

0.1

In [11]:
0.1 + 100. - 100.

0.09999999999999432

#### 3) Machine precision and cumulative errors

Consider the integral $$I_n = \int_{0}^1 \frac{x^n}{x + 10}dx$$

1. Without computing $I_n$, find its limit.
2. Compute $I_0$ and find a recurrence formula between $I_{n+1}$ and $I_n$
3. If we were to compute $I_n$ recursively, would that algorithm be stable numerically ?

1.

$$ \lim I_n = 0$$

2.

$$ I_0 = \int_0^1 \frac{1}{x+10} dx = [\ln (x+10)]_0^1 = \ln(\frac{11}{10})$$

$$ I_{n+1} = \frac{1}{n+1} - 10 * I_n$$



In [14]:
def integral(i0, n):
    ii = i0
    for k in range(n):
        ii = (1/(k+1)) - 10 * ii
        print(ii)
    return ii

In [15]:
integral(np.log(11/10), 20)

0.04689820195675065
0.031017980432493486
0.023153529008398455
0.018464709916015454
0.015352900839845474
0.013137658268211921
0.011480560175023635
0.010194398249763648
0.009167128613474629
0.008328713865253717
0.007621952256553738
0.00711381076779595
0.005784969245117427
0.013578878977397152
-0.06912212310730485
0.7537212310730486
-7.478388781318721
74.83944336874276
-748.3418021084802
7483.468021084803


7483.468021084803

4. Replace 10 in the integral with a constant A > 1. Given a machine precision variable $\varepsilon$, how can we set the number of iterations $n$ based on a desired cumulative error E ?

In [17]:
print(np.finfo(float).eps) 

2.220446049250313e-16


**Independent questions:**

**Q5.** Write a piece of code that can find $\varepsilon$ numerically. 



In [21]:
e = 1e-12
print(1+e == 1)

e = 1e-24
print(1 + e == 1)



False
True


**Q6.** Given what you know now, how should you test if two numbers or arrays are equal ? 

In [20]:
x = 0.2 + 0.1

y = 0.3

np.abs(x - y) / np.max(np.abs([x, y])) < 1e-10

True

#### 4) Logsum-exp trick
Consider a classification model with 4 classes. We are modeling the probablity of a sample being in class $k$ with: $$p_k = \frac{ exp(w_k)}{\sum_{i=1}^{4} exp(w_i)}$$

where $w$ are the weights of a neural net.
1. Why does this model make sense ? 
2. Given the example $w = [-20, -1, 0, 1000]$, it is obvious which class this sample should correspond to. Compute the prediction probabilities using the function below for this particular example. Explain.

In [22]:
def predict(w):
    p = np.exp(w)
    p /= p.sum()
    return p

w = np.array([10, -1, 40, 2, 1000])
predict(w)

/var/folders/wz/mdj42wg170j7vt8s9ljnyn580000gn/T/ipykernel_24859/437721929.py:2: RuntimeWarning: overflow encountered in exp
  p = np.exp(w)
/var/folders/wz/mdj42wg170j7vt8s9ljnyn580000gn/T/ipykernel_24859/437721929.py:3: RuntimeWarning: invalid value encountered in divide
  p /= p.sum()


array([ 0.,  0.,  0.,  0., nan])

3. Even if we assume that $exp(w_k)$ do not overflow, computing the normalizing sum can cause problems if the number of labels is too large. After showing the following statement, propose a method to modify `predict` in order to avoid overflow errors:
$$ \forall c \in \mathbb{R} \qquad log\left(\sum_{k=1}^K exp(w_k + c)\right) =  c + log\left(\sum_{k=1}^K exp(w_k)\right) $$ 

In [ ]:
def lse(w):
	c = np.max(w)
	wp = w - c
    return ll

def predict_stable(w):
    return 

In [ ]:
predict_stable(w)

4. Generate random weight vectors with `np.random.randn(K)` and test that both functions return the same probabilities. Test it with the scipy `logsumexp`: 

In [ ]:
from scipy.special import logsumexp
